# MOF-74NiでのH2O分子の吸着と拡散

******************
### Updates
2022/05/16  - minor revisions mainly to accomodate calculator and optimizer setups
******************

この事例ではMOF-74（aka CPO-27）をもちいて水分子の吸着と拡散挙動についてPFPを適用して観察してみます。
MOF-74は金属がNi、Co、Zn、Mgなど様々なバージョンがありますが、いずれもDOBDCでBridgeされて1次元のnanoporousな構造を有しており、金属部分がむき出しになっている状態です。ここに水分子や他の小分子が吸着することが可能で吸着材、分離膜材料、触媒材料として期待されています。

MOFは一般に多様な元素が混在しており、かつ疎な構造であるためDFT計算では計算負荷が高く、古典力場を用いた手法ではパラメーターの導出に多大な労力と計算リソースが必要になります。PFPを用いることによってこれらの課題を超えて分子動力学計算やモンテカルロ計算など様々な計算が短時間で実行可能になります。

本計算事例は以下のPFP論文及びMatlantisウェブサイトでも公開されています。<br/>
※ 計算結果自体は、論文執筆時と異なるPFP versionを使用しているため、異なっています。

 - [Towards universal neural network potential for material discovery applicable to arbitrary combination of 45 elements | Nature Communications](https://www.nature.com/articles/s41467-022-30687-9)
 - [MOF-74NiにおけるH2O分子の吸着と拡散挙動 | MATLANTIS](https://matlantis.com/ja/calculation/adsorption-and-dynamics-of-h2o-molecules-in-mof-74ni)

まず、必要なライブラリをまとめてインポートしておきます。

In [1]:
# Generatl import statements

# PFP　
import pfp_api_client
print(f"pfp_api_client: {pfp_api_client.__version__}")
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator

estimator = Estimator(calc_mode='CRYSTAL_PLUS_D3',model_version='v2.0.0') #DFT-D3
calculator = ASECalculator(estimator)

#print("estimator : ",estimator)
print("calc_mode: ",estimator.calc_mode)
#print("calculator : ",calculator)

# ASE
from ase.io import read,write,cif,gaussian,vasp,PickleTrajectory,xyz
from ase import Atoms, Atom
from ase.constraints import FixAtoms 
from ase.build import bulk
from ase.visualize import view
from ase.build import surface
from ase.constraints import StrainFilter,ExpCellFilter,UnitCellFilter
from ase.optimize import LBFGS,FIRE,MDMin
from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md import MDLogger
from ase import units

# Other external packages
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import json
from IPython.display import Image, display_png
import nglview as nv
import ipywidgets as widgets

pfp_api_client: 1.3.1
calc_mode:  CRYSTAL_PLUS_D3


In [2]:
import os

indir = Path("input")
outdir = Path("output")
os.makedirs(outdir, exist_ok=True)

必要な構造（水分子、MOF-74、MOF-74と水分子の吸着構造）のそれぞれの最適化を行います。

In [3]:
#水分子のみ
h2o = read(indir/"H2O.POSCAR")
h2o.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
opt = LBFGS(h2o)
opt.run(fmax=fmax)
write(indir/"H2O_opt.POSCAR",h2o)
epot_h2o = h2o.get_potential_energy() 
epot_h2o

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 02:59:46      -10.027575*       0.3033
LBFGS:    1 02:59:46      -10.029338*       0.1671
LBFGS:    2 02:59:46      -10.031338*       0.1400
LBFGS:    3 02:59:46      -10.031676*       0.0411
LBFGS:    4 02:59:47      -10.031692*       0.0129
LBFGS:    5 02:59:47      -10.031693*       0.0006


-10.031692870771481

MOF-74とMOF-74に水分子が吸着した構造は最適化にExpCellFilterを用いてCellサイズと原子座標の両方を同時に最適化してみます。

In [4]:
#MOF-74
# "Ni", "Co", "Zn", "Mg" can be used.
metal = "Ni"

mof = read(indir/f"MOF-74{metal}.POSCAR")
mof.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
ef = UnitCellFilter(mof)
opt = LBFGS(ef)
opt.run(fmax=fmax)
write(indir/f"MOF-74{metal}_opt.POSCAR",mof)
epot_mof = mof.get_potential_energy() 
epot_mof

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 02:59:50     -938.647949*       1.0633
LBFGS:    1 02:59:50     -939.093579*       1.0454
LBFGS:    2 02:59:50     -939.512021*       0.9515
LBFGS:    3 02:59:50     -940.521684*       1.1416
LBFGS:    4 02:59:50     -940.874757*       1.0059
LBFGS:    5 02:59:50     -941.182328*       0.6007
LBFGS:    6 02:59:51     -941.284316*       0.4653
LBFGS:    7 02:59:51     -941.375801*       0.3667
LBFGS:    8 02:59:51     -941.448024*       0.3279
LBFGS:    9 02:59:51     -941.500932*       0.2432
LBFGS:   10 02:59:51     -941.571761*       0.2713
LBFGS:   11 02:59:51     -941.618445*       0.2840
LBFGS:   12 02:59:51     -941.675080*       0.2440
LBFGS:   13 02:59:51     -941.728283*       0.2243
LBFGS:   14 02:59:51     -941.781856*       0.2137
LBFGS:   15 02:59:51     -941.826965*       0.2244
LBFGS:   16 02:59:52     -941.869761*       0.2273
LBFGS:   17 02:59:52     -941.9087

-943.5864729162331

最適化後の構造が問題ないか確認します。

In [5]:
from ase.visualize import view

v = view(mof, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

次いでMOF-74に水分子が吸着した構造です。

In [6]:
#MOF-74と水分子の吸着構造
mof_and_h2o = read(indir/f"MOF-74{metal}_and_18H2O.POSCAR")
mof_and_h2o.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
ef = UnitCellFilter(mof_and_h2o)
opt = LBFGS(ef)
opt.run(fmax=fmax)

write(outdir/f"MOF-74{metal}_and_18H2O_opt.POSCAR", mof_and_h2o)
epot_mof_and_h2o = mof_and_h2o.get_potential_energy() 
epot_mof_and_h2o

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 03:00:18    -1133.914285*       0.5052
LBFGS:    1 03:00:18    -1134.013468*       0.4086
LBFGS:    2 03:00:18    -1134.077339*       0.4079
LBFGS:    3 03:00:18    -1134.289808*       0.6908
LBFGS:    4 03:00:19    -1134.387603*       0.5115
LBFGS:    5 03:00:19    -1134.472873*       0.4921
LBFGS:    6 03:00:19    -1134.513699*       0.4860
LBFGS:    7 03:00:19    -1134.559250*       0.3791
LBFGS:    8 03:00:19    -1134.606035*       0.3446
LBFGS:    9 03:00:19    -1134.634641*       0.3590
LBFGS:   10 03:00:19    -1134.684847*       0.4360
LBFGS:   11 03:00:19    -1134.656430*       0.9924
LBFGS:   12 03:00:20    -1134.781595*       0.4803
LBFGS:   13 03:00:20    -1134.835221*       0.5050
LBFGS:   14 03:00:20    -1134.874078*       0.2865
LBFGS:   15 03:00:20    -1134.946548*       0.2882
LBFGS:   16 03:00:20    -1134.998818*       0.3902
LBFGS:   17 03:00:20    -1135.0508

-1138.3690276986385

In [7]:
from ase.visualize import view

view(mof_and_h2o,viewer="ngl")
v.view.add_representation("ball+stick")
display(v)

ここで得られたエネルギーの値から吸着Energyを求めてみます。各Ni原子に一つ水分子が吸着しているので、この構造の中では18個の水分子が存在します。

In [9]:
# Updated data:
(epot_mof_and_h2o - epot_mof - 18*epot_h2o)/18.0

-0.789560172695488

このPFP versionでは、水分子一つ当たり-0.71 eVという値が得られました(旧versionでは、`-0.67`eV)。

In [10]:
# Updated data:

lat1 = mof.cell.cellpar()
lat2 = mof_and_h2o.cell.cellpar()
v1 = mof.get_volume()
v2 = mof_and_h2o.get_volume()
dv = (v2-v1)/v1*100

print("格子定数と角度：")
print("　　MOFのみ：   {0:.3f} {1:.3f} {2:.3f} {3:.3f} {4:.3f} {5:.3f}".format(*lat1))
print("　　MOFと18H2O：{0:.3f} {1:.3f} {2:.3f} {3:.3f} {4:.3f} {5:.3f}".format(*lat2))
print("体積：")
print("　　MOFのみ：   {0:.2f}".format(v1))
print("　　MOFと18H2O：{0:.2f}".format(v2))
print("　　変化率：    {0:+.2f} %".format(dv))

格子定数と角度：
　　MOFのみ：   25.859 25.859 6.671 89.996 90.003 119.985
　　MOFと18H2O：26.025 26.068 6.854 90.002 89.966 119.986
体積：
　　MOFのみ：   3863.58
　　MOFと18H2O：4027.25
　　変化率：    +4.24 %


旧version:

```
格子定数と角度：
　　MOFのみ：   25.682 25.678 6.796 89.998 90.005 119.982
　　MOFと18H2O：26.071 26.116 6.952 90.006 90.008 120.021
体積：
　　MOFのみ：   3882.23
　　MOFと18H2O：4098.45
　　変化率：    +5.57 %
```

水分子吸着前後で比較すると、水分子が吸着されると格子定数が1~2%程度大きくなり、体積も>4%増加することが分かる。

次に吸着分子の挙動を分子動力学シミュレーションで追ってみます。

In [12]:
T = 573
num_md_steps  = 200000 
num_int_print = 200
time_step = 0.5 * units.fs

MaxwellBoltzmannDistribution(mof_and_h2o, T*units.kB)
Stationary(mof_and_h2o)

dyn = Langevin(mof_and_h2o, time_step, T * units.kB, 0.02, logfile=outdir/f"md_MOF-74{metal}_and_H2O_573K_nvt.log")

def printenergy(a=mof_and_h2o):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    #a.wrap()
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

dyn.attach(printenergy, interval=num_int_print)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory(outdir/f"md_MOF-74{metal}_and_H2O_573K_nvt.traj", 'w', mof_and_h2o)
dyn.attach(traj.write, interval=num_int_print)

# Now run the dynamics
printenergy()
dyn.run(num_md_steps)

/home/jovyan/.local/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Energy per atom: Epot = -5.270eV  Ekin = 0.075eV (T=581K)  Etot = -5.195eV
Energy per atom: Epot = -5.270eV  Ekin = 0.075eV (T=581K)  Etot = -5.195eV
Energy per atom: Epot = -5.225eV  Ekin = 0.043eV (T=333K)  Etot = -5.182eV
Energy per atom: Epot = -5.218eV  Ekin = 0.046eV (T=355K)  Etot = -5.173eV
Energy per atom: Epot = -5.215eV  Ekin = 0.051eV (T=391K)  Etot = -5.165eV
Energy per atom: Epot = -5.213eV  Ekin = 0.054eV (T=417K)  Etot = -5.159eV
Energy per atom: Epot = -5.211eV  Ekin = 0.058eV (T=448K)  Etot = -5.153eV
Energy per atom: Epot = -5.209eV  Ekin = 0.058eV (T=453K)  Etot = -5.150eV
Energy per atom: Epot = -5.204eV  Ekin = 0.065eV (T=507K)  Etot = -5.139eV
Energy per atom: Epot = -5.202eV  Ekin = 0.064eV (T=494K)  Etot = -5.138eV
Energy per atom: Epot = -5.199eV  Ekin = 0.065eV (T=503K)  Etot = -5.134eV
Energy per atom: Epot = -5.203eV  Ekin = 0.067eV (T=517K)  Etot = -5.136eV
Energy per atom: Epot = -5.204eV  Ekin = 0.071eV (T=553K)  Etot = -5.132eV
Energy per atom: Epot = -

True